<a href="https://colab.research.google.com/github/sugiyama404/ReinforcementLearningForGymOrAtari/blob/main/Control/MultiProcessByMultiCarRacing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt update && apt install xvfb
!git clone https://github.com/igilitschenski/multi_car_racing.git
!pip install ./multi_car_racing
!pip install -U imgaug==0.2.7 gym-notebook-wrapper

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:13 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB

In [2]:
import gym
from gym import wrappers
import gym_multi_car_racing
import numpy as np
import gnwrapper

import concurrent.futures
import os

In [3]:
class FlatMultiDisplayWrapper(gym.Wrapper):
    def __init__(self, env, nrows=1, ncols=1, empty_cell=0):
        super().__init__(env)
        self.nrows = nrows
        self.ncols = ncols
        self.empty_cell = empty_cell

    def render(self, mode="human", **kwargs):
        stack_images = super().render(mode, **kwargs)
        nstack = stack_images.shape[0]
        frame_shape = stack_images.shape[1:]

        if len(frame_shape) >= 2:
            self.flat_shape = list(frame_shape)
            self.flat_shape[0] = self.flat_shape[0]*self.nrows
            self.flat_shape[1] = self.flat_shape[1]*self.ncols
            frame = np.full(self.flat_shape, self.empty_cell, dtype=np.uint8)

            for i in range(min(nstack, self.nrows*self.ncols)):
                row = i // self.ncols
                col = i % self.ncols
                frame[row*frame_shape[0]:(row+1)*frame_shape[0],
                      col*frame_shape[1]:(col+1)*frame_shape[1]] = stack_images[i]
        else:
            # たまに、空の画像を stack した画像が返ってくる。
            frame = np.full(self.flat_shape, self.empty_cell, dtype=np.uint8)

        return frame

    def step(self, action):
        obs, rew, done, info = super().step(action)
        # Reward が gym.Env の仕様を満たしていないので変換する。元の値はinfoに入れておく
        info["original_reward"] = rew
        return obs, rew.sum(), done, info

In [4]:
actions = np.array([[ 0, 0, 0],  # [0]: straight
                    [ 0, 1, 0],  # [1]: acceleration
                    [ 0, 0, 1],  # [2]: decelerate
                    [ 1, 0, 0],  # [3]: Turn right
                    [-1, 0, 0]]) # [4]: Turn left

def worker():
    os.environ["CUDA_VISIBLE_DEVICES"] = ""
    action = np.random.randint(5)
    action = actions[action]
    return action

In [5]:
env = gym.make("MultiCarRacing-v0", num_agents=4, direction='CCW',
        use_random_direction=True, backwards_flag=True, h_ratio=0.25,
        use_ego_color=False)

env = FlatMultiDisplayWrapper(env, nrows=2, ncols=2)
env = gnwrapper.Monitor(env, './', force = True)

obs = env.reset()
done = False
total_reward = 0

while not done:
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as executor:
        f1 = executor.submit(worker)
        f2 = executor.submit(worker)
        f3 = executor.submit(worker)
        f4 = executor.submit(worker)
    arr = [f1.result(),f2.result(),f3.result(),f4.result()]
    obs, reward, done, info = env.step(arr)
    total_reward += reward

print("individual scores:", total_reward)

env.display()

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Track generation: 1182..1474 -> 292-tiles track
individual scores: -343.2989690721605


'openaigym.video.0.84.video000000.mp4'